In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import LeaguesModel

leagues_model = LeaguesModel()

leagues_model.load_counties_geojson()

In [ ]:
leagues_model.load_counties_data()

In [ ]:
leagues_model.load_leagues(["NFL", "NBA", "MLB", "NHL", "MLS"])

In [ ]:
leagues_model.calculate_distances() 

In [ ]:
leagues_model.compute_shares()  

In [ ]:
leagues_model.compute_output_dataframe()      

## NFL

In [ ]:
leagues_model.heatmap_counties("NFL")

leagues_model.render_map("NFL")[0]

## MLB

In [ ]:
leagues_model.heatmap_counties("MLB")

leagues_model.render_map("MLB")[0]

## NBA

In [ ]:
leagues_model.heatmap_counties("NBA")

leagues_model.render_map("NBA")[0]

## NHL

In [ ]:
leagues_model.heatmap_counties("NHL")

leagues_model.render_map("NHL")[0]

## MLS

In [ ]:
leagues_model.heatmap_counties("MLS")

leagues_model.render_map("MLS")[0]